In [1]:
import os
import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from models.non_constant_diffusivity import NonConstantDiffusivityNeuralNetwork
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/model_paper')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/model_paper


In [3]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear/non_linear.pkl


In [4]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cuda


In [6]:
# Tratamiento de los datos para dividirlos en train y test
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
y_train = TensOps(torch.tensor(dataset['y_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_train = TensOps(torch.tensor(dataset['k_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_train = TensOps(torch.tensor(dataset['f_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_np = X_train
y_np = y_train.values
K_np = K_train.values
f_np = f_train.values

X_train_np, X_test_np, y_train_np, y_test_np, K_train_np, K_test_np, f_train_np, f_test_np = train_test_split(X_np, y_np, K_np, f_np, test_size=0.2, random_state=42)

X_train = X_train_np.to(device)
X_test = X_test_np.to(device)

y_train = TensOps(y_train_np.to(device), space_dimension=y_train.space_dim, contravariance=0, covariance=0)
y_test = TensOps(y_test_np.to(device), space_dimension=y_train.space_dim, contravariance=0, covariance=0)

K_train = TensOps(K_train_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)
K_test = TensOps(K_test_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)

f_train = TensOps(f_train_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)
f_test = TensOps(f_test_np.to(device), space_dimension=K_train.space_dim, contravariance=0, covariance=0)

In [7]:
# Arquitectura del modelo
input_shape = X_train[0].shape  # [1, 10, 8]
hidden1_dim = 150
hidden2_dim = 150
output_shape = y_train.values[0].shape  # [1, 10, 10]

In [8]:
# Se carga el modelo y el optimizador
model = NonConstantDiffusivityNeuralNetwork(input_shape, hidden1_dim, hidden2_dim, output_shape).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 10001

batch_size = 64
n_checkpoints = 100

train_loop(model, optimizer, n_checkpoints,
           X_train, y_train, X_test, y_test, f_train, f_test,
           D=D, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=device,
        )

Start training
Epoch 0, Train loss: 4.800e+07, Test loss: 3.730e+07, MSE(e): 4.790e+00, MSE(pi1): 7.446e+00, MSE(pi2): 2.091e+00, MSE(pi3): 2.474e-01
Epoch 100, Train loss: 2.751e+04, Test loss: 2.505e+04, MSE(e): 1.867e-03, MSE(pi1): 2.673e-01, MSE(pi2): 1.200e-03, MSE(pi3): 6.164e-02
Epoch 200, Train loss: 5.887e+03, Test loss: 5.100e+03, MSE(e): 4.681e-04, MSE(pi1): 7.559e-02, MSE(pi2): 3.230e-04, MSE(pi3): 4.496e-03
Epoch 300, Train loss: 7.087e+03, Test loss: 4.187e+03, MSE(e): 6.206e-04, MSE(pi1): 6.209e-02, MSE(pi2): 3.596e-04, MSE(pi3): 2.593e-03
Epoch 400, Train loss: 5.566e+03, Test loss: 3.775e+03, MSE(e): 4.924e-04, MSE(pi1): 4.123e-02, MSE(pi2): 2.599e-04, MSE(pi3): 2.296e-03
Epoch 500, Train loss: 4.996e+03, Test loss: 3.354e+03, MSE(e): 4.403e-04, MSE(pi1): 3.881e-02, MSE(pi2): 2.342e-04, MSE(pi3): 2.039e-03
Epoch 600, Train loss: 3.299e+03, Test loss: 3.529e+03, MSE(e): 2.589e-04, MSE(pi1): 5.221e-02, MSE(pi2): 1.409e-04, MSE(pi3): 1.878e-03
Epoch 700, Train loss: 1.154

In [ ]:
# Parametros de entrenamiento
start_epoch = 10000
n_epochs = 110001

batch_size = 64
n_checkpoints = 100

second_lr = 3e-5

train_loop(model, optimizer, n_checkpoints,
           X_train, y_train, X_test, y_test, f_train, f_test,
           D=D, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=device,
        )

Start training
Starting from a checkpoint. Epoch 10000.
Epoch 10000, Train loss: 1.530e+02, Test loss: 1.714e+02, MSE(e): 4.862e-06, MSE(pi1): 3.315e-03, MSE(pi2): 2.445e-06, MSE(pi3): 7.124e-04
Epoch 10100, Train loss: 1.934e+02, Test loss: 1.650e+02, MSE(e): 7.075e-06, MSE(pi1): 5.581e-03, MSE(pi2): 3.365e-06, MSE(pi3): 6.689e-04
Epoch 10200, Train loss: 1.542e+02, Test loss: 1.903e+02, MSE(e): 3.392e-06, MSE(pi1): 4.427e-03, MSE(pi2): 1.527e-06, MSE(pi3): 7.596e-04
Epoch 10300, Train loss: 1.280e+02, Test loss: 1.449e+02, MSE(e): 2.225e-06, MSE(pi1): 3.672e-03, MSE(pi2): 1.069e-06, MSE(pi3): 6.900e-04
Epoch 10400, Train loss: 1.698e+02, Test loss: 1.573e+02, MSE(e): 4.589e-06, MSE(pi1): 5.057e-03, MSE(pi2): 2.562e-06, MSE(pi3): 7.329e-04
Epoch 10500, Train loss: 1.447e+02, Test loss: 1.983e+02, MSE(e): 3.097e-06, MSE(pi1): 3.892e-03, MSE(pi2): 1.309e-06, MSE(pi3): 7.477e-04
Epoch 10600, Train loss: 1.154e+02, Test loss: 1.261e+02, MSE(e): 1.521e-06, MSE(pi1): 2.804e-03, MSE(pi2): 6.